In [21]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [22]:
df = pd.read_csv('df.csv', index_col=0)
cidades = pd.read_csv('cidades.csv', index_col=0)

In [23]:
df.rename(columns={'id_municipio_nascimento': 'id_municipio'}, inplace=True)

In [24]:
idMunicipios = cidades['id_municipio'].to_list()
flt = df['id_municipio'].isin(idMunicipios)
df = df[flt]

df['id_municipio'] = df['id_municipio'].astype('int')
df['id_municipio'] = df['id_municipio'].astype('str')

In [25]:
cidades['id_municipio'] = cidades['id_municipio'].astype(int)
cidades['id_municipio'] = cidades['id_municipio'].astype(str)

In [26]:
df = df.merge(cidades[['id_municipio', 'nome_municipio']], on='id_municipio', how='left')

In [27]:
df = df.dropna(subset=['apgar1'])
flt = df['apgar1'] > 10
df = df[~flt]

# replace values with more descriptive ones
df = df.replace({'sexo': {1: 'Masculino', 2: 'Feminino', 'M': 'Masculino', 'F': 'Feminino'},
                 'tipo_parto': {1: 'Vaginal', 2: 'Cesárea', 9: 'Ignorado'}})

In [28]:
years = [1998, 1999, 2000, 2001, 2002, 2003, 2004,
         2005, 2006, 2007, 2008, 2009, 2010, 2011,
         2012, 2013, 2014, 2015, 2016, 2017, 2018,
         2019, 2020]

In [29]:
# do not remove missing values for hora_nascimento
df_filtered = df.dropna(subset=['peso', 'apgar5', 'idade_mae', 'tipo_parto'])

In [ ]:
for year in years:
    
    flt = df['ano'] == year
    dfFiltered = df_filtered[flt]
    dfFiltered.to_csv(f'../Nascidos-Vivos-Viz/general/general{year}.csv', index=False)
    
    apgarMean1 = dfFiltered.groupby(['id_municipio', 'nome_municipio', 'ano'])['apgar1'].mean().reset_index(name = 'meanApgar1')
    apgarMean1['meanApgar1'] = apgarMean1['meanApgar1'].round(2)
    apgarMean1.to_csv(f'../Nascidos-Vivos-Viz/mean/apgar1Mean{year}.csv', index=False)

In [ ]:
apgarMean1 = df_filtered.groupby(['id_municipio', 'nome_municipio', 'ano'])['apgar1'].mean().reset_index(name = 'meanApgar1')
pivot = pd.pivot_table(apgarMean1, index='id_municipio', columns = 'ano', values='meanApgar1').reset_index().round(2)
pivot.to_csv('../Nascidos-Vivos-Viz/meanPivot/apgarMean1Pivoted.csv', index=False)